## Memory 

LLM이 가져오는 프롬프트에 기록들을 넣는건 "우리의 일"이라는 것 ! 

Langchain은 Off-the-shelf Chain이 있는데 그건 자동으로 LLM으로부터 응답값을 가져오고 memory를 업데이트 한다는 것임. 

Prompt와 memory 클래스는 바뀌지 않았고, prompt는 MessagesPlaceholder를 가지고 있고 history라는 변수명을 가지고 있는 값에 가지고 있는 메세지 값을 넣어주길 기대합니다.



In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("My name is nico")

content='Hello Nico! How can I assist you today?'


In [3]:
invoke_chain("I want to know abour korea")

content='Sure, I can provide you with information about Korea. Are you interested in learning about the culture, history, tourist attractions, or something specific about Korea? Let me know how I can help you.'


In [4]:
invoke_chain("What is my name ?")

content='Your name is Nico. How can I assist you further, Nico?'
